In [1]:
import numpy as np
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [39]:
%run D:\\python-code\train_test_split.py

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [40]:
%run D:\\python-code\LinearRegression.py

In [41]:
lin_reg = LinearRegression()
%time lin_reg.fit_normal(X_train, y_train)
lin_reg.score(X_test, y_test)

Wall time: 0 ns


0.8129802602658533

### 使用梯度下降法

In [42]:
lin_reg2 = LinearRegression()
lin_reg2.fit_gd(X_train, y_train)

D:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
D:\python-code\LinearRegression.py:27: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
D:\python-code\LinearRegression.py:45: RuntimeWarning: invalid value encountered in double_scalars
  if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):


LinearRegression()

可以看到报错了，其中有一个关键字 overflow。

In [43]:
lin_reg2.coef_

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

从上面结果来看，还是溢出了。这是因为此时是一个真实的数据。

下面我们来看 X_train 前十行的数据

In [44]:
X_train[:10, :]

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00],
       [1.15172e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        5.70100e+00, 9.50000e+01, 3.78720e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.58770e+02, 1.83500e+01],
       [6.58800e-02, 0.00000e+00, 2.46000e+00, 0.00000e+00, 4.88000e-01,
        7.76500e+00, 8.33000e+01, 2.74100e+00, 3.00000e+00, 1.93000e+02,
        1.78000e+01, 3.95560e+02, 7.56000e+00],
       [2.49800e-02, 0.00000e+

可以看到数据规模是不一样的，有些数据是零点几，有些可以达到几百这样一个维度。那么面对这样的一个数据，我们实际最终求到的梯度很有可能结果也是那么大的，我们使用默认的η最终形成的步长还是太大，使得我们这个梯度下降法的过程是不收敛的。

那么为了验证我们的假设，我们还是使用 fit_gd 函数进行训练，但是我们传入的 eta=0.000001，小了非常多倍。

In [53]:
lin_reg2.fit_gd(X_train, y_train, eta=0.000001)# 此时不报错

LinearRegression()

In [50]:
lin_reg2.score(X_test, y_test)

nan

可以看到此时得到的R方值为0.27，很显然，此时我们使用梯度下降法所找到的θ还没有达到我们损失函数的最小值。很有可能是因为我们的η太小了，导致每一步行径都非常小，所以我们需要在梯度下降法中用更多的循环次数才能找到损失函数的最小值。

所以我们再进行一次训练，手动将循环次数 n_iters 100万

In [54]:
%time lin_reg2.fit_gd(X_train, y_train, eta=0.000001, n_iters=1e6)

Wall time: 320 ms


LinearRegression()

In [55]:
lin_reg2.score(X_test, y_test)

0.27556634853389206

&emsp;&emsp;换句话说，我们增加了循环次数，但是结果还是没有达到我们损失函数达到最小值，其实我们的循环次数还需要更多，这样显然太耗时了，那么对于这种情况应该怎么办呢？

&emsp;&emsp;其实我们之前已经分析出来了，之所以出现这种情况，是因为这些数据整体不在一个规模上，其实解决的的方式我们之前也学习过，就是**数据的归一化**。

&emsp;&emsp;我们之前使用正规方程解来解决线性回归算法的时候，其实不需要进行数据的归一化，这是因为我们将线性回归模型的求解过程整体变成了一个公式的计算，那么在这个公式的计算中，牵扯这种中间搜索的过程比较少，所以我们不需要进行数据归一化。

&emsp;&emsp;可是当我们使用梯度下降法的时候，就变得不一样了，由于我们有η这个变量，所以首先会出现一个问题，如果我们最终这些数值不在一个维度上，将会影响我们梯度的结果，而梯度的结果再乘以η是我们真正每一次走的步长，这个步长就有可能或者太大，或者太小。如果太大，会导致结果不收敛，就像之前使用默认的η得到的结果那样，如果太小，又会导致我们的搜索过程太慢，就像我们上面实验的那样。但是如果我们将所有的数据进行归一化，那么这个问题就完全解决了。


&emsp;&emsp;下面，我们来具体实验一下。

### 使用梯度下降法前进行数据归一化

In [56]:
from sklearn.preprocessing import StandardScaler

In [58]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [59]:
X_train_standard = standardScaler.transform(X_train)

In [63]:
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard, y_train)

Wall time: 163 ms


LinearRegression()

In [61]:
X_test_stardard = standardScaler.transform(X_test)

In [62]:
lin_reg3.score(X_test_stardard, y_test)

0.8129880620122235

&emsp;&emsp;从结果可以看出，和我们使用正规方程解得到的结果是一致的，说明我们找到了这个损失函数的最小值。与此同时，速度是非常快的。这就是数据归一化的威力。

&emsp;&emsp;但是我们发现，使用梯度下降法需要花费163ms，而使用正规方程解是0ns，那么梯度下降法有什么优势呢？

&emsp;&emsp;在这里，我们再举一个例子。

### 梯度下降法的优势

#####  设计一个虚拟的样本数据

In [66]:
m = 1000 # 样本数
n = 5000 # 样本特征

big_X = np.random.normal(size=(m, n))#随机化的正态分布，所以自送生成的数据已经归一化了

# 随即生成n+1个theta的取值
true_theta = np.random.uniform(0.0, 100.0, size=n+1)
big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0, 10., size=m)

In [67]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y) # 使用正规方程解

Wall time: 2.9 s


LinearRegression()

In [68]:
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y) # 使用梯度下降法

Wall time: 2.06 s


LinearRegression()

&emsp;&emsp;从上面结果可以看出，使用正规方程解耗时2.9s，使用梯度下降法为2.06s，梯度下降法稍微快一些。如果我们加大我们的特征数，梯度下降法的优势会更加明显一些。

&emsp;&emsp;这就是所说的对于正规方程解，它处理的是 m x n 这样大的矩阵进行非常多的乘法运算，所以这个矩阵比较大的时候，其实我们的正规方程法相应的就要更耗时一些。

&emsp;&emsp;不过在我们所举的例子中，样本数量是小于样本特征数的。这是因为我们现在所使用的梯度下降法的这个公式，在计算梯度的时候，我们要让每一个样本都参与计算，这使得当样本数量比较大的时候，我们计算这个梯度其实相应的也比较慢，其实这也有一种改进的方式，就是所谓的**随机梯度下降法**。